In [1]:
import itertools
import numpy as np
import scipy.special as ss

In [6]:
p_vals = np.arange(1,20)
n_vals = np.arange(1,20)
e_vals = np.arange(33)

attack_length = 2
e_lists = list(itertools.product(*[e_vals]*attack_length))

# cartesian product of possible values of p, n, (e_{ell-1}, e_ell)
vectors = list(itertools.product(*[p_vals, n_vals, e_lists]))

In [7]:
def calcDelayDifference(p, n, e_list):
    e_ell = e_list[1]
    return 40 * (p - n) + 8 * max(24-e_ell, 0) - 8 * max(e_ell-8, 0)

def calcRewardDifference(p, e_list):
    block_reward = 16 * (1/(p+1) + 0.8 + 0.2 * (e_list[1] / 32) - 1)
    endorse_reward = 2 * e_list[0] * (1/(p+1) - 1)
    return block_reward + endorse_reward

def calcProb(alpha, p, n, e_list):
    binoms = [ss.binom(32, e) for e in e_list]
    return np.product(binoms) * np.power(alpha, n + sum(e_list) + 1) * np.power(1- alpha,65  + p - sum(e_list))

In [8]:
# minutes in a year
C = 365 * 24 * 60
for alpha in np.arange(0.1, 0.45, 0.05):
    probabilites = []
    values = []
    for p, n, e_list in vectors:
        reward = calcRewardDifference(p, e_list)
        # if feasible and profitable
        if (calcDelayDifference(p, n, e_list) <= 0) and (reward > 0):
            prob = calcProb(alpha, p, n, e_list)
            probabilites.append(prob)
            values.append(prob * reward)
    # expected reward per minute is alpha * (block reward + total endorse reward)
    expected_reward = alpha * ( 16 +  2 * 32 )
    percent_increase = 100 * sum(values) / expected_reward 
    
    print(alpha, sum(probabilites) * C, sum(values) * C, percent_increase)
    print(sum(probabilites))

0.1 0.2194549402046173 0.5373286518697472 1.2778934833279758e-05
4.175322302218746e-07
0.15000000000000002 4.18691885483562 7.9388106170355055 0.00012586901663234882
7.965979556384361e-06
0.20000000000000004 28.593946574699782 45.944812931313606 0.0005463376727943492
5.440248587271648e-05
0.25000000000000006 98.08946291515483 130.5772418345174 0.0012421731529158808
0.00018662378789032502
0.30000000000000004 188.46917704995806 203.82074442575762 0.0016157783519292045
0.00035857910397632813
0.3500000000000001 209.63579897072066 190.1816476419517 0.0012922758183976928
0.00039885045466271055
0.40000000000000013 136.9647193970565 110.79044726051377 0.000658714131828587
0.00026058736567172093
